This code provides a foundation for the Sobol indices using the Saltelli method. Pleae make edits where noted

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# STUDENTS: Define the function f
def f(x):
    # Placeholder for the cubic function; replace with actual implementation
    return np.sum(x**3)

# STUDENTS: Define upper and lower bounds (students need to provide values)
UB = np.array([])  # Upper bounds
LB = np.array([])  # Lower bounds
num_par = len(UB)

# Assume that all parameters are uniform
parameter_names = ['$q_1$', '$q_2$', '$q_3$', '$q_4$']

M = 10000  # Number of samples for Sobol
A = np.random.uniform(LB, UB, (M, num_par))
B = np.random.uniform(LB, UB, (M, num_par))

# Generate C matrices
C_cell = []
for i in range(num_par):
    C = A.copy()
    C[:, i] = B[:, i]
    C_cell.append(C)

# Initialize the output storage structure
f_A = np.zeros(M)  # A matrix
f_B = np.zeros(M)  # B matrix
f_C = np.zeros((M, num_par))  # C matrices

for j in range(M):
    f_A[j] = f(A[j, :])
    f_B[j] = f(B[j, :])
    for k in range(num_par):
        f_C[j, k] = f(C_cell[k][j, :])

y_D = np.concatenate([f_A, f_B])
var_Y = np.var(y_D)
S_i = np.zeros(num_par)
ST_i = np.zeros(num_par)

for k in range(num_par):
    S_i[k] = np.sum(f_B * (f_C[:, k] - f_A)) / (var_Y * M)
    ST_i[k] = np.sum((f_A - f_C[:, k]) ** 2) / (var_Y * (2 * M))

# Bar plot for first-order and total indices
plt.figure()
plt.bar(range(num_par), S_i, alpha=0.6, label='First-order')
plt.bar(range(num_par), ST_i, alpha=0.6, label='Total')
plt.legend()
plt.title('Integral QoI')
plt.xticks(range(num_par), parameter_names)
plt.show()

# Generate density plots
plt.figure(figsize=(10, 8))
for k in range(num_par):
    plt.subplot(2, 2, k + 1)
    fA_kde = gaussian_kde(f_A)
    fC_kde = gaussian_kde(f_C[:, k])
    x_vals = np.linspace(min(f_A.min(), f_C[:, k].min()), max(f_A.max(), f_C[:, k].max()), 1000)
    plt.plot(x_vals, fA_kde(x_vals), 'b', label='A', linewidth=2)
    plt.plot(x_vals, fC_kde(x_vals), 'r', label='C', linewidth=2)
    plt.legend()
    plt.title(parameter_names[k])
    plt.grid(True)
plt.tight_layout()
plt.show()
